# Introduction


## What is RAG?

**Retriever augmented generation** (**RAG**) is a system that improves the response generated by a LLM in two ways:

* First, the information is retrieved from a dataset that is stored in vector database; the query is used to perform similarity search in the documents stored in the vector database.
* Second, by restraining the context provided to the LLM to content that is similar with the initial query, stored in the vector database, we can reduce significantly (or even eliminate) LLM's halucinations, since the answer is provided from the context of the stored documents.

An important advantage of this approach is that we do not need to fine-tune the LLM with our custom data; instead, the data is ingested (cleaned, transformed, chunked, and indexed in the vector database).

## Procedure

* We start by using directly a prompt with questions for the LLM using a transformers pipeline.
* We test the pipeline with few questions.
* We then ingest the text of the Letters of Ion Ghica in a vector database that will serve as the retriever. The vector database used is ChromaDB.
* We assemble the retriever-generation system. For this, we compose a prompt with the system message instructing the LLM how to use the initial query and the context retrieved from the vector database; in the user message we give the query as input.
* We test with the same questions we tried without the RAG system (directly prompting the LLM) and more questions.

# Install and include packages

In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install langchain
!pip install sentence-transformers
!pip install chromadb

Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not currently take into account all the 

In [2]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.9 MB/s eta 0:00:00


In [3]:
from time import time
from IPython.display import display, Markdown
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForCausalLM

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

2024-05-27 18:15:47.964834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 18:15:47.964961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 18:15:48.098435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Prepare the model pipeline

In [4]:
pipe = pipeline("text-generation", 
                model="/kaggle/input/openllm-ro-q4b/transformers/7b-instruct-q4b/1/RoLlama2-7b-Instruct-v1-q4b",
               kwargs=['_load_in_4bit'])

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Test the text generator function

In [5]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipe(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipe.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an AI assistant designed to answer simple questions from the works of Ion Ghica.
Please restrict your answer to the exact question asked.
"""

# Run few tests with the text generator

In [6]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [7]:
response = query_model(
    system_message,
    user_message="Cine a fost dascalul Chiosea?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.




**<font color='red'>Question:</font>** Cine a fost dascalul Chiosea? 

**<font color='green'>Answer:</font>**  Dascalul Chiosea a fost un dascal (învățător) din secolul al 19-lea, care a lucrat în satul Chiosea din județul Prahova, România. El a fost cunoscut pentru abilităț 

**<font color='magenta'>Total time:</font>** 7.13 sec.

In [8]:
response = query_model(
    system_message,
    user_message="Cine a fost Marin Serghiescu?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cine a fost Marin Serghiescu? 

**<font color='green'>Answer:</font>**  Marin Serghiescu a fost un politician român, membru al Partidului Conservator, care a deținut funcția de ministru al agriculturii și domeniilor în timpul domniei lui Alexandru Ioan Cuza.  

**<font color='magenta'>Total time:</font>** 5.34 sec.

In [9]:
response = query_model(
    system_message,
    user_message="Cum il chema pe fratele lui Teodor Diamant?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cum il chema pe fratele lui Teodor Diamant? 

**<font color='green'>Answer:</font>**  Teodor Diamant a avut un frate pe nume Ion Ghica.  

**<font color='magenta'>Total time:</font>** 2.21 sec.

In [10]:
response = query_model(
    system_message,
    user_message="La cine pranzaste Ion Ghica pe timpul calatoriei sale la Iasi?",
    temperature=0.1,
    max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** La cine pranzaste Ion Ghica pe timpul calatoriei sale la Iasi? 

**<font color='green'>Answer:</font>**  Ion Ghica a luat prânzul cu Ion C. Brătianu în timpul călătoriei sale la Iași.  

**<font color='magenta'>Total time:</font>** 3.67 sec.

In [11]:
response = query_model(
    system_message,
    user_message="Unde s-a născut Bărzof?",
     temperature=0.1,
     max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Unde s-a născut Bărzof? 

**<font color='green'>Answer:</font>**  Bărzof s-a născut în satul Bărzău din județul Brăila.  

**<font color='magenta'>Total time:</font>** 2.95 sec.

In [12]:
response = query_model(
    system_message,
    user_message="Cât de mare era curtea banului Ghica?",
     temperature=0.1,
     max_length=64)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Cât de mare era curtea banului Ghica? 

**<font color='green'>Answer:</font>**  Curtea banului Ghica era de aproximativ 100 de hectare.  

**<font color='magenta'>Total time:</font>** 2.2 sec.

The answers are somehow accurate, with some exceptions. Let's check if we can get better answers with a RAG system where we index the text of the letters in a vector database that we will use as the retriever part of the system.

# Retrieval preparation

We will create now the retrieval functionality. We will use a ChromaDB instance for vector database. We will ingest the text from Ion Ghica work, "Letters to Vasile Alecsandri" in Romanian. Here we have the comlete text of Ion Ghica most known work.

## Ingest the text

### Load the document

In [13]:
loader = PyPDFLoader("/kaggle/input/scrisori-ctre-vasile-alecsandri-de-ion-ghica/Scrisori_catre_Vasile_Alecsandri.pdf")
documents = loader.load()

### Split the text and prepare the embeddings

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=800, 
            chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)
# create a vectorstore database
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Ingest the data in ChromaDB

In [15]:
vector_db = Chroma.from_documents(documents=all_splits, 
                                embedding=embeddings, 
                                persist_directory="chroma_db")

### Define the retriever

In [16]:
retriever = vector_db.as_retriever()

## Define the query

We define functions to generate the system prompt, user prompt and to get the context documents.

In [17]:
def get_system_prompt(document_text):
    system_prompt=f"""
        INSTRUCTIUNI:
            Vă rog să răspundeți la întrebările utilizatorilor folosind DOCUMENT-ul furnizat.
            Limitați răspunsul la faptele consemnate în DOCUMENT.
            Dacă documentul nu conține faptele pentru a răspunde la ÎNTREBARE menționați aceasta 
            în mod clar și nu returnați nimic.
        DOCUMENT:
            {document_text}
        """
    return system_prompt

In [18]:
def get_user_prompt(query):
    user_prompt =f"""
                ÎNTREBARE:
                    {query}
                RĂSPUNS:
                """
    return user_prompt

In [19]:
def get_documents(query):
    # generate the answer
    docs = retriever.get_relevant_documents(query)
    data = ""
    for item in list(docs):
        data += item.page_content
    
    return data

We define a colorized text function for RAG.

In [20]:
def colorize_text_rag(text):
    for word, color in zip(["Question", "Prompt", "Answer", "Total time", "Context"], 
                           ["blue", "magenta", "red", "darkblue", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [21]:
template = "\n\nQuestion:\n{question}\n\nPrompt:\n{prompt}\n\nAnswer:\n{answer}\n\nTotal time:\n{total_time}\n\nContext:\n{context}"

def rag_query(query,
         temperature=0.1,
         max_length=1024,
         show_context=False):
    start_time = time()
    
    # Retriever
    context = get_documents(query)
    
    # Augmented generation
    system_message = get_system_prompt(context)
    user_message = get_user_prompt(query)

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipe(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipe.model.config.eos_token_id
    )

    answer = sequences[0]['generated_text']
    end_time = time()
    total_time = f"{round(end_time-start_time, 2)} sec."
    if show_context:
        return template.format(question=query, prompt=user_message, answer=answer, total_time=total_time,context=context)
    else:
        return template.format(question=query, prompt=user_message, answer=answer, total_time=total_time,context=None)


# Perform tests with RAG

In [22]:
response = rag_query("Cum il chema pe fratele lui Theodor Diamant?",
     temperature=0.1,
     max_length=64)
display(Markdown(colorize_text_rag(response)))

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




**<font color='blue'>Question:</font>**
Cum il chema pe fratele lui Theodor Diamant?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cum il chema pe fratele lui Theodor Diamant?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 
                RĂSPUNS:
                Nicu Bălcescu 

**<font color='darkblue'>Total time:</font>**
4.93 sec.

**<font color='green'>Context:</font>**
None

In [23]:
response = rag_query("Cum il chema pe fratele lui Theodor Diamant?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cum il chema pe fratele lui Theodor Diamant?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cum il chema pe fratele lui Theodor Diamant?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Theodor Diamant nu are un frate menționat în document. 

**<font color='darkblue'>Total time:</font>**
4.28 sec.

**<font color='green'>Context:</font>**
l'empereur Alexandre et l'empereur Nicolas avait rendu si
riche et si prospere, votre administration l'a reduit a la plus
complete misere: il arrive tous les jours a Petersbour g des
plaintes et des lamentations de la part des habitants d'Ismail
et de Bolgrad'.
Am raportat atunci aceste cuvinte domnitorului Cuza,
zicându-i că ele îmi făcuseră mie mare impresiune și că mi
se păreau un rău prognostic pentru viitorul Basarabiei.blagoveștenie, preobrejenie, blagoslovenie, bogoslov ,
zlatauz, pristol și denie.
Am asistat la înființarea Societății literare și societăței
'Frăția' și la răscularea de la 1848.
Am văzut îndoita alegere a lui Cuza-vodă, proclamarea
Unirei Principatelor și lovitura de stat.
Am asistat la noaptea de 1 1 fevruarie la plebiscitul lui vodă
Carol de Hohenzollern, apoi la declararea independenței
României și a regatului și la încoronarea regelui Carol I.552Voi urma în epistola următoare vorbindu-ți despre rolul și
activitatea lui Nicu Bălcescu în timpul cât a trăit proscris,
de la anul 1848 până la 1853.
Am auzit și am văzut pe mai mulți atribuindu-și lor , numai
lor, tot meritul revoluțiunii de la 1848, și mi-am zis adesea:
Căci nu trăiește Nicu Bălcescu, că le-ar spune cu pana lui
cea ageră, în stilul său cel pătrunzător , că deșteptarea de la
1848 nu a fost opera unui singur om, nici a unui grup sau a
unui partid, ci a fost opera simțului național al României
întregi, a fost rezultatul a trei generațiuni; le-ar spune că
acea mare revoluțiune nu putea fi oprită de nimeni și că ea
s-ar fi făcut chiar în lipsa fiecăruia dintr -acei cari au sau au
avut cutezanța a-i revendica paternitatea. Cugetătorul și515generațiunii care se ridică le zic: 'Fiți patrioți și modești ca
Alexandrescu, și când voiți să știți cum se glorifică faptele
cele mari, citiți odele și elegiile lui; când voiți să știți cum
se râde de ambițioșii de rând și cum se biciuiește viciul,
citiți satirele și fabulele lui Alexandrescu'. Dotat cu un
talent mare, el și-a iubit țara, s-a respectat pe sine și a lăsat
în memoria noastră operele spiritului și ale inimei sale.

In [24]:
response = rag_query("Cine a fost dascalul Chiosea?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cine a fost dascalul Chiosea?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cine a fost dascalul Chiosea?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Dascalul Chiosea a fost un om blând, modest și devotat. 

**<font color='darkblue'>Total time:</font>**
3.93 sec.

**<font color='green'>Context:</font>**
reprezenta, că puii corbului erau nevinovații de români, și
lebăda, omul care dă sfaturi bune, și ceruse pedeapsa
cutezătorului autor:
 
A zis ...asemenea nebunie, acela ar fi numai vrednic de plîns, iar nu
de temut. Folosiți-vă de binefacerile ce societatea noastră285unul din ucenicii săi; a trăit mult cu dânsul la Munich și la
Paris.
Nu am cunoscut un om mai blând, mai modest și mai
devotat decât dânsul. Filantrop și umanitar , era din neamul
acelora din cari au ieșit apostolii, martirii și sfinții.
O fi având el, oare, o piatră pe mormânt?547Guvernul începu a fi neliniștit și cugeta a face arestări; dar ,
viind aga Iancu Manu în sfatul domnesc cu propunerea să
aresteze trei inși, și anume, pe C.A. Rosetti, pe Grigorie
Scarlat Grădișteanu și pe Ion Ghica, se scoală Costache
Cornescu, logofătul bisericesc (ministru cultelor), și zice că
era de prisos să aresteze pe nepotul său Ruset, căci nu mai
putea fi primejdios, fiind gata să plece la Paris. Atunci un
alt membru al sfatului, Grigorie Grădișteanu, zice că el nu
admite escepțiuni și că ori să se aresteze câte trei sau nici
unul, căci dacă era să se facă favoruri, atunci va cere și el
escepțiune pentru nepotul său, Grigorie Sc. Grădișteanu,
căruia și lui i-ar fi mai plăcut a se plimba pe ulițele
Parisului decât a sta închis la gros.

In [25]:
response = rag_query("Cine l-a luat pe Marin Serghiescu cântăreț la Biserica Negustorilor?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cine l-a luat pe Marin Serghiescu cântăreț la Biserica Negustorilor?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cine l-a luat pe Marin Serghiescu cântăreț la Biserica Negustorilor?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Ștefan Cantu 

**<font color='darkblue'>Total time:</font>**
3.32 sec.

**<font color='green'>Context:</font>**
forței.
„Părinții bisericei — zice Cantu — proclamau libertatea
credințelor în tot timpul persecuțiunii religiei creștine,
îndată însă ce a încetat acea persecuțiune și biserica a
devenit autoritate, au început la rîndul lor a vorbi în numele
puterii.” Aceste două curente ale libertății și al autorității le
găsim în luptă în toate societățile; și chiar în cele mai
înaintate în civilizațiune găsim o luptă neîncetată între
justiție și putere, fiecare posedînd o acțiune puternică
asupra societății lor și biruind cînd una, cînd alta.547Guvernul începu a fi neliniștit și cugeta a face arestări; dar ,
viind aga Iancu Manu în sfatul domnesc cu propunerea să
aresteze trei inși, și anume, pe C.A. Rosetti, pe Grigorie
Scarlat Grădișteanu și pe Ion Ghica, se scoală Costache
Cornescu, logofătul bisericesc (ministru cultelor), și zice că
era de prisos să aresteze pe nepotul său Ruset, căci nu mai
putea fi primejdios, fiind gata să plece la Paris. Atunci un
alt membru al sfatului, Grigorie Grădișteanu, zice că el nu
admite escepțiuni și că ori să se aresteze câte trei sau nici
unul, căci dacă era să se facă favoruri, atunci va cere și el
escepțiune pentru nepotul său, Grigorie Sc. Grădișteanu,
căruia și lui i-ar fi mai plăcut a se plimba pe ulițele
Parisului decât a sta închis la gros.Principate. Apostolii cei mai activi erau în Iași T eodor
Negris, secretarul fostului domn Calimah, Anagnostopolu șinevasta lui, după ce l-a despuiat în pielea goală, l-a uns cu
cătran din creștet până în tălpi, i-a pus o pereche de coarne
pe cap, l-a legat răstignit de mâni pe un drug, i-a legat un
căluș în gură și l-a luat în șfichiul biciului pe pod, de fugea
lumea de dânsul ca de Ucigă-l toaca, încât nici chiar slugile
lui nu au voit să-l primească în casă. A doua zi l-a găsit
vizitiul ascuns în ieslele din grajd. Pe un alt cuconaș
bărbatul l-a uns cu miere pe piele și l-a lăsat o zi întreagă
prada muștelor și viespelor .

In [26]:
response = rag_query("Din cine era coborâtor Ioniță Sandulache Sturdza?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




**<font color='blue'>Question:</font>**
Din cine era coborâtor Ioniță Sandulache Sturdza?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Din cine era coborâtor Ioniță Sandulache Sturdza?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Ioniță Sandulache Sturdza era un descendent al lui Vlad Țepeș. 

**<font color='darkblue'>Total time:</font>**
4.2 sec.

**<font color='green'>Context:</font>**
cunoască păsurile țării, cerea să se trimită la Constantinopol
o deputațiune cu care guvernul otoman să se poată înțelege.
În deputațiunile trimise de la Iași și de la București era și
Ioniță Sandulache Sturdza, un coborâtor al lui Vlad Țepeș și
Grigorie Dimitrie Ghica, nepot de frate decăpitatului
Grigorie Alexandru Ghica al Moldovei.
După mai multe conferințe cu vizirul și cu reis-efendi,
conferințe cari au durat mai bine de două luni, pe la iulie
sus-menționații doi boieri au fost anunțați că sunt numiți522Domnul V asile Alecsandri atribuie unui alt tânăr plin de
talent conceperea și compunerea în limba franceză a acelui
cap d-operă, Alecu Russu, răpit și el, ca și Bălcescu, de
timpuriu familiei și patriei sale.
Iată ce știu eu despre acea scriere epică din care, încă pe la
anul 1847, Bălcescu îmi citise mai multe fragmente și pe
care, mai târziu, ne-a citit-o întreagă, în cabinetul
generalului Mavru, unde se afla și Laurianu și Bolliac,
susținând că găsise acel manuscript la un călugăr . Iată cu ce
ocaziune ne-a făcut el acea citire:
Generalul Mavru, filolog și numismat de frunte, era de mai
mulți ani în relațiune cu Laurianu și cu Bolliac, cu unul
pentru studiul de medalii antice și schimburi de dublete, cu
celalalt pentru citirea și transcripțiunea de inscripțiuni38Când eram pe la 1829 la Craiova, l-am văzut de gât cu
generalul Roth, juca cărți cu miile de galbeni, căci era peste
proviant.asupra lui T udor, îi pun în fiare mâinile și picioarele și-l duc
în tabăra lui Ipsilante, unde-l aștepta Caravia, Cavaleropolo

In [27]:
response = rag_query("Unde a crescut Bârzof?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Unde a crescut Bârzof?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Unde a crescut Bârzof?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Bârzof a crescut în mahalaua Izvorul, pe strada Brezoianu, prin Dudescu, la Antim și în fața bisericii Doamna Bălașa. 

**<font color='darkblue'>Total time:</font>**
8.18 sec.

**<font color='green'>Context:</font>**
367ani cu luntrea prin mahalaua Izvorul, pe strada Brezoianu,
prin Dudescu, pe la Antim și pe dinaintea bisericei Doamna
Bălașa? Astăzi, grădina, curtea, paraclisul și casa banului
Scarlat Ghica s-au împărțit în bucățele pătrate și
numerotate, s-au tras printre dânsele strade și s-au clădit
case și căscioare.
În sfaturile boierilor , numele eroului de la Marengo era de
câtăva vreme foarte adesea pomenit; speranțele tuturor erau
întoarse către Franța și către minunile ce se auzea că se
petreceau acolo. Deși unii dintre boieri, ca Pană Filipescu,
ca banul Costache Ghica și alții, se gândeau la ce era să zică
de dânșii împăratul Alexandru al Rusiei sau împăratul
Leopold al Austriei când or afla că au trimis soli la desculții
de francezi, nimeni nu îndrăznea să iasă din vorba banului73de-i cânta cântece de lume. Când Grigorie Ghica s-a făcut
domn, Nicolae Alexandrescu a ajuns cafegi-bașa.
Pe Marin Ser ghiescu l-a luat Alecache V illara cântăreț la
Biserica Negustorilor , l-a primblat pe dânsul pe la
Petersbur g, cu Regulamentul; dar mai pe urmă, molipsindu-
se de boala patriotismului, a fost arestat cu Mitică Filipescu,
cu Nicolae Bălcescu și cu T elegescu, trimis la ocnă, unde
și-a spășit păcatele șapte ani, până când a venit de l-a scos
revoluția din patruzeci și opt, ca să-i procure dulcile fericiri
ale unei proscripțiuni de zece ani. El a fost cunoscut mai
mult sub numele de Marin Naționalu. V odă Cuza îl iubea, îl
avea adesea la masă și-l mângâia cu numele de moș Marin.
Când a venit în București dascălul Lazăr , băieții de la338„Și crezi că Redclif fe ar susține numirea aceasta?” Eu îi
răspund că eram sigur de sprijinul ambasadorului
Engliterei, dacă alteța-sa mă autoriză să-i vorbesc ca din
parte-i.
Astfel autorizat, am văzut pe lordul Redclif fe chiar în seara
aceea, și el a chemat îndată pe întîiul său secretar Alison și
l-a pornit la București, ca să se înțeleagă cu fostul <lomn
Alexandru Ghica, care în curînd a fost numit caimacam în
Valahia deodată cu T oderiță Balș în Moldova; acesta era
susținut de baronul Prokesch, după recomandațiunea
contelui austriac Goedel.
În vara anului 1857, comisarii europeni se adunase la
Constantinopol și porneau unul după altul la București, într -
o zi primisem vizita principelui Labanof, care venise să mă
vadă la Hissar , împreună cu comisarul francez T alleyrand și51 1ce am făcut acum vreo câțiva ani unui poet de curte de
atunci, pe care-l povățuiam cu versurile:
Ia-ți nădragii de atlas
De-ți fă steagul la Parnas!
Apoi ce fac eu cu acele versuri când voi îmbrăca nădragii
de atlas?'
Mai târziu a fost numit director la Departamentul Credinței,
pe când moșiile mănăstirilor închinate și neînchinate se
arendau cum da Domnul, post important și ambiționat de
toți acei cari căuta să facă averi mari. El, care l-a ocupat
atâți ani, a ieșit de acolo sărac precum intrase.
În anul 1853 se auzea de război. Alexandrescu, ca și Nicu
Bălcescu au avut totdeauna credința că România numai prin
arme se putea ridica la rangul ce i se cuvine. Era vorba ca o
armată auxiliară franceză să vie să ocupe valea Dunărei de

In [28]:
response = rag_query("Cât de mare era curtea banului Ghica?",
     temperature=0.1,
     max_length=64,
    show_context=True)
display(Markdown(colorize_text_rag(response)))



**<font color='blue'>Question:</font>**
Cât de mare era curtea banului Ghica?

**<font color='magenta'>Prompt:</font>**

                ÎNTREBARE:
                    Cât de mare era curtea banului Ghica?
                RĂSPUNS:
                

**<font color='red'>Answer:</font>**
 Curtea banului Ghica era cât o moșie de mare. 

**<font color='darkblue'>Total time:</font>**
4.15 sec.

**<font color='green'>Context:</font>**
364mai închine, că și nemții, și muscalii, dacă băteau pe turci,
tot pe turta lor trăgeau spuza.
La cererea d-a se da țării domn pământean, pe bătrânul
Părscoveanu, Poarta răspunsese trimițând pe un alt fanariot
în locul lui Alexandru Moruz, pe Mihai Suțu.
În seara de Mucenici, boierii cei mai colțați erau adunați la
sfat la banul Dumitrache Ghica, fratele lui Grigorie-vodă,
asasinat pentru împotrivirea ce făcuse la darea Bucovinei.
Erau adunați la banul bătrânul, cum îi ziceau ca să-l
deosebească de doi fii ai săi, Costache și Scarlat, cari
amândoi ajunseseră tot bani mari și ei.
Curtea banului Ghica era cât o moșie de mare; începea din
capătul despre apus al Podului Gârlei, devale de Zlătari,
mergea în jos pe malul apei până în dreptul caselormai bănoasă, era de-a ataca caravana de la Mecca și a jăfui
cămila care ducea darurile sultanului la mormîntul
profetului.367ani cu luntrea prin mahalaua Izvorul, pe strada Brezoianu,
prin Dudescu, pe la Antim și pe dinaintea bisericei Doamna
Bălașa? Astăzi, grădina, curtea, paraclisul și casa banului
Scarlat Ghica s-au împărțit în bucățele pătrate și
numerotate, s-au tras printre dânsele strade și s-au clădit
case și căscioare.
În sfaturile boierilor , numele eroului de la Marengo era de
câtăva vreme foarte adesea pomenit; speranțele tuturor erau
întoarse către Franța și către minunile ce se auzea că se
petreceau acolo. Deși unii dintre boieri, ca Pană Filipescu,
ca banul Costache Ghica și alții, se gândeau la ce era să zică
de dânșii împăratul Alexandru al Rusiei sau împăratul
Leopold al Austriei când or afla că au trimis soli la desculții
de francezi, nimeni nu îndrăznea să iasă din vorba banuluiboierimii, dară inima lui îl trăgea mai mult către lumea
învățată, către profesori, către oamenii de litere și către

# Conclusions

We implemented a RAG system for Romanian. The LLM used was a quantized version of the Romanian LLM.

Using RAG, we can focus the answers on the exact set of documents we are targeting. We could see that, for some of the questions, we get very "to the point" answers in the case of RAG system while using the direct prompting the LLM method we get also some halucinations.

The answers to the questions are delivered in relatively short time, around 2-3 seconds most of them.

I suggest to you to download a copy of the Ion Ghica's work and check few of the facts in the text. 

